# seg

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}<style>"))

In [2]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import random
import keras.backend as K
from keras.models import load_model
import datetime

DATA_SHAPE = 224
val_images = np.load('/home/mc16/pre_data/val_image.npy')
val_labels = np.load('/home/mc16/pre_data/val_label_w3.npy')
test1_images = np.load('/home/mc16/pre_data/test1_image.npy')

In [3]:
import tensorflow as tf
from keras import layers, Input, Model, Sequential, optimizers
from keras.layers import Reshape, Merge, Lambda, Add, Subtract, dot
from keras.layers import Layer, Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.regularizers import l2
from keras.engine import Layer
from keras.utils import multi_gpu_model, np_utils
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
GPU_MEMORY_FRACTION = 1.0

In [4]:
GPU_MEMORY_FRACTION = 0.3
def config_keras_backend(fraction):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = fraction 
    sess = tf.Session(config=config)
    K.set_session(sess)

# detect points for one image
def detect_points(img, model, shape):
    img = cv2.resize(img, (shape,shape))
    img = np.expand_dims(img,axis=0)
    points_logist = model.predict(img)
    points = np.argmax(points_logist, axis=-1)
    return points[0]

config_keras_backend(GPU_MEMORY_FRACTION)
model = load_model('/home/mc16/model3/unet0615.h5')
#model = load_model('/home/mc16/model7_segcrf/seg_two.h5')

/home/mc16/.local/lib/python2.7/site-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
def detect_test_list(model, shape):
    test_set_path = '/data/mc_data/MLDC/data/test1/images/' 
    test_md5_paths = open('/home/mc16/test1_list.txt','r').readlines()
    points_list = np.zeros((len(test_md5_paths), shape, shape), np.uint8)
    i = 0
    for path in tqdm(test_md5_paths):
        md5 = path.split('.')[0]
        img = plt.imread(test_set_path + md5 +'.jpg')
        points_logist = detect_points(img, model, shape)
        points_list[i] = points_logist
        i = i + 1
    return points_list

def detect_val_points(val_images, model, shape):
    points_list = np.zeros((len(val_images), shape, shape), np.uint8)
    i = 0
    for image in tqdm(val_images):
        points_list[i] = detect_points(image, model, shape)
        i = i + 1
    return points_list

val_seg = detect_val_points(val_images, model, DATA_SHAPE)
# test1_seg = detect_test_list(model, DATA_SHAPE)

In [7]:
from keras.preprocessing import image
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import compute_unary, create_pairwise_bilateral, create_pairwise_gaussian, unary_from_softmax
import warnings
warnings.simplefilter("ignore", np.RankWarning)

def crf_image(img, model, gdims=(2,3), bdims=(20,100), iteras=1):
    img = cv2.resize(img, (DATA_SHAPE, DATA_SHAPE))
    score = model.predict(np.expand_dims(img,axis=0))

    softmax = score[0].transpose((2, 0, 1)) # [channel,shap,shape]
    unary = unary_from_softmax(softmax)
    unary = np.ascontiguousarray(unary)#(2,n)
    crf = dcrf.DenseCRF(img.shape[0] * img.shape[1], 2)
    crf.setUnaryEnergy(unary)

    feats = create_pairwise_gaussian(sdims=gdims, shape=img.shape[:2])
    crf.addPairwiseEnergy(feats, compat=3, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

    feats = create_pairwise_bilateral(sdims=bdims, schan=(20, 20, 20), img=img, chdim=2)
    crf.addPairwiseEnergy(feats, compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

    Q = crf.inference(iteras)
    
    res = np.argmax(Q, axis=0).reshape((img.shape[0], img.shape[1]))
    return res    

def cut_shit(mask, num, frac=0.7):
    cut_mask = np.array(mask)
    for j in range(224):
        for i in range(223-num):
            if np.sum(cut_mask[j, i:i+num]) > num*frac:
                cut_L = i + int(num * (1 - frac * 0.75))
                cut_R = i + int(num * (1 - frac * 0.25))
                cut_mask[j, cut_L:cut_R] = 0
    return cut_mask

In [8]:
CRF_GDIMS = (3,3)
CRF_BDIMS = (50,50)
CRF_ITERAS = 1
CUT_WIDTH = 30
CUT_FRAC = 0.5
def get_crf_list(image_list):
    crf_list = []
    for img in tqdm(image_list):
        crf = crf_image(img, model, gdims=CRF_GDIMS, bdims=CRF_BDIMS, iteras=CRF_ITERAS)
        cut = cut_shit(crf, CUT_WIDTH, CUT_FRAC)
        crf_list.append(cut)
    return crf_list
val_crf = get_crf_list(val_images)

100%|██████████| 2015/2015 [12:43<00:00,  2.64it/s]


In [ ]:
def get_cluster_list(crf_list):
    cluster_list = []
    for crf in tqdm(crf_list):
        point = matrix_to_point(cut)
        cluster = get_cluster_points(point, EPS = CLUSTER_EPS, MIN_SAMPLES = 3, minN=CLUSTER_MINN)
        cluster_list.append(cluster)
    return cluster_list